# Genetic Algorithm Study with a tutorial

## Initial population set

In [54]:
import math
import random

def init_generation(chromo_len, pop_size, generation,
                    next_generation, fitness, selector_prob):
    '''Initial first generation by random'''
    n = 2 ** chromo_len -1  # max value of chromosome
    for i in range(pop_size):
        generation.append((random.randint(0, n), random.randint(0, n)))
        next_generation.append((0, 0))
        fitness.append(0)
        selector_prob.append(0)   

## Population definition

In [55]:
age = 0                 # the count of generation
pop_size = 30           # totoal population of a generation
gen_max = 50            # max amount of generation
chromo_len = 25        # length of chromosome
cross_prob = 0.8        # probability of crossover
mutat_prob = 0.1        # probability of mutation

generation = []         # current generation of individuals
next_generation = []    # next generation of individuals
fitness = []            # fitness value of each individual in current generation
selector_prob = []      # probability of each individual to be selected

# the best individual of that generation
elitist = {'chromosome':(0, 0), 'fitness':0, 'age':0}

# Initial generation params
init_generation(chromo_len, pop_size, generation,
                next_generation, fitness, selector_prob)

## Define encoding and decoding function

In [56]:
def decode(interval, chromosome, chromo_len):
    '''Project chromosome to feasible interval'''
    delta = interval[1] - interval[0]
    ratio = float(2 ** chromo_len -1)
    return (interval[0] + chromosome * delta / ratio)    

## Define fitness function

In [62]:
def fitness_func(chromo_x, chromo_y):
    '''Calculate the fitness value of a chromosome pair'''
    interval = [-10.0, 10.0]
    x = decode(interval, chromo_x, chromo_len)
    y = decode(interval, chromo_y, chromo_len)
    nom = lambda x,y: math.sin(math.sqrt(x*x + y*y)) ** 2 - 0.5
    denom = lambda x,y: (1 + 0.001 * (x*x + y*y)) ** 2
    f = lambda x,y: 0.5 - nom(x,y) / denom(x,y)
    return f(x,y)

def evaluate(pop_size, fitness, generation, selector_prob):
    '''Evalutate the selection probability of each individual'''
    for i in range(pop_size):
        fitness[i] = fitness_func(generation[i][0], generation[i][1])
    
    total = sum(fitness)
    for i in range(pop_size):
        selector_prob[i] = fitness[i] / total
    
    # For Roulette Selection
    for i in range(1, pop_size):
        selector_prob[i] += selector_prob[i-1] 
        

## Define genetic operation

In [66]:
def select(selector_prob):
    t = random.random()
    i = 0 
    for p in selector_prob:
        if p > t:
            break
        i += 1
    return i

def cross (chromo_len, cross_prob, chrom1, chrom2):
    p = random.random ()
    n = 2 ** chromo_len -1
    if chrom1 != chrom2 and p < cross_prob:
        t = random.randint (1, chromo_len - 1)
        #print bin(chrom1),bin(chrom2)
        mask = n << t
        (r1, r2) = (chrom1 & mask, chrom2 & mask)
        #print mask,bin(mask)
        #print bin(r1),bin(r2)
        mask = n >> (chromo_len - t)
        (l1, l2) = (chrom1 & mask, chrom2 & mask)
        #print mask,bin(mask)
        #print bin(l1), bin(l2)
        
        (chrom1, chrom2) = (r1 + l2, r2 + l1)
    return (chrom1, chrom2)

    
def mutate(chromo, chromo_len):
    p = random.random()
    mutat_prob = 1
    if p < mutat_prob:
        t = random.randint(1, chromo_len)
        mask1 = 1 << (t - 1)
        mask2 = chromo & mask1
        if mask2 > 0:
            chromo = chromo & (~mask2)
        else:
            chromo = chromo ^ mask1
    return chromo

## Evolution process

In [67]:
for k in range(gen_max):
    # Evaluate selection prob of current generation
    evaluate(pop_size, fitness, generation, selector_prob)
    
    # Evolve with genetic operations
    i = 0
    while True:
        # Select individual
        id1 = select(selector_prob)
        id2 = select(selector_prob)
        (id1_x, id1_y) = (generation[id1][0], generation[id1][1])
        (id2_x, id2_y) = (generation[id2][0], generation[id2][1])
        
        # Perform cross
        (id1_x, id2_x) = cross(chromo_len, cross_prob, id1_x, id2_x)
        (id1_y, id2_y) = cross(chromo_len, cross_prob, id1_y, id2_y)
        
        # Perform mutate
        id1_val = (mutate(id1_x, chromo_len), mutate(id1_y, chromo_len))
        id2_val = (mutate(id2_x, chromo_len), mutate(id2_y, chromo_len))
         
        # Generate next generation    
        next_generation[i] = id1_val
        next_generation[i + 1] = id2_val
        i += 2
        
        # Check termination condition
        if i >= pop_size:
            break
    
    # Replace the current generation with next one
    generation = list(next_generation)
    print (k, max(fitness), sum(fitness)/pop_size, min(fitness)) 

(0, 0.9884934500378899, 0.685662185422906, 0.02562632508772189)
(1, 0.9899524298802628, 0.6995285129249985, 0.028803147788303618)
(2, 0.9807479136717451, 0.7365419908077772, 0.041823916962629715)
(3, 0.9872525641322631, 0.7050299794666909, 0.025546239622170674)
(4, 0.9892576851034987, 0.7284594167161312, 0.05773912221943289)
(5, 0.9819217372129203, 0.7500094859179209, 0.0679961263958469)
(6, 0.9870928794355894, 0.7376802563226986, 0.07702596541426276)
(7, 0.9872015160786689, 0.7048947302408148, 0.022706615078970593)
(8, 0.9870569342811868, 0.723771955450505, 0.07491531424408088)
(9, 0.9902775341583623, 0.7645175871424303, 0.07588826792484882)
(10, 0.9900542424474348, 0.7340151283545605, 0.06333896784226495)
(11, 0.9902818598567149, 0.8079302642713077, 0.05594272788596272)
(12, 0.9901560203268471, 0.7042881895866462, 0.039176466159353995)
(13, 0.9901585659030369, 0.7038763720079709, 0.030077303211614004)
(14, 0.9901279617125118, 0.665073049817197, 0.03720001713242532)
(15, 0.99021335301